In [ ]:
# Import the modules
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# 1. ETL Workflow

## Extract:

In [ ]:
# Read in the CSV file as a Pandas DataFrame
file_path = Path('dataset/spotify_youtube.csv')
spotify_youtube_df = pd.read_csv(file_path)

# # Display sample data
spotify_youtube_df.head()

## Transform:

In [ ]:
# Drop unnecessary first row
spotify_youtube_df = spotify_youtube_df.drop(spotify_youtube_df.columns[0], axis=1)

# Check dataframe
spotify_youtube_df.head() 

In [ ]:
# Convert column headers to lowercase for consistency and readability
spotify_youtube_df.columns = spotify_youtube_df.columns.str.lower()

# Check dataframe
spotify_youtube_df.head()

In [ ]:
# Engineer new features
# Create engagement to measure total song activity
spotify_youtube_df['engagement'] = spotify_youtube_df['views'] + spotify_youtube_df['stream']

# Create content ratio to measure YouTube popularity for each song
spotify_youtube_df['youtube popularity'] = spotify_youtube_df['likes'] / spotify_youtube_df['views']

# Check dataframe
spotify_youtube_df.head()

In [ ]:
# Transform album_type, licensed, and official_video columns to numerical values for machine learning compatability
# Replace values in 'album_type' with numerical representations
spotify_youtube_df['album_type'] = spotify_youtube_df['album_type'].replace({'album': 0, 'single': 1, 'other': 2})

# Convert 'licensed' and 'official_video' from boolean to integers
# 1 = True (licensed/official video), 0 = False (not licensed/not official video)
spotify_youtube_df['licensed'] = spotify_youtube_df['licensed'].fillna(0).astype(int)
spotify_youtube_df['official_video'] = spotify_youtube_df['official_video'].fillna(0).astype(int)
spotify_youtube_df

In [ ]:
# Summary Statistics
spotify_youtube_df.describe()

Due to the counts of the different columns being different, we wish to eliminate any rows that may contain NaN values

In [ ]:
spotify_youtube_df_cleaned = spotify_youtube_df.dropna(how='any')
spotify_youtube_df_cleaned

In [ ]:
spotify_youtube_df_cleaned.describe()

Because of the large disparity in values between features, we have decided that we should scale the values in the millions down.

In [ ]:
# Isolate compatible columns
feature_set = spotify_youtube_df_cleaned[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'views', 'likes', 'comments', 'licensed', 'official_video', 'stream', 'engagement', 'youtube popularity']]

# Initialize scaler
scaler = StandardScaler()

# Scale values
scaled_features = scaler.fit_transform(feature_set)
scaled_df = pd.DataFrame(scaled_features, columns=feature_set.columns)
scaled_df

## EDA: Visualizations

In [ ]:
# Plot histograms
spotify_youtube_df_cleaned.hist(bins=30, figsize=(20, 15))
plt.tight_layout()
plt.show()

In [ ]:
# Plot histograms
scaled_df.hist(bins=30, figsize=(20, 15))
plt.tight_layout()
plt.show()

### Correlation Heat Map

In [ ]:
# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(scaled_df.corr(), annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Heat Map with Correlation Values')
plt.show()

### Observations
- Positive Correlations
  - energy and loudness
  - valnce and danceabiity
- Negative Correlations
  - acousticness and energy
  - acousticness and loudness
  - instrumentalness and loudness

### Conclusions
- We can combine features to avoid redundancy and overfitting:
  - energy and loudness = power
  - valence and danceability = uplift_factor

In [ ]:
# Create scatter plot
plt.figure(figsize=(10, 6))
sizes = spotify_youtube_df_cleaned['likes'] / 10000  # Scale down 'likes' for better visualization

plt.scatter(spotify_youtube_df_cleaned['stream'], spotify_youtube_df_cleaned['views'], alpha=0.5, label='Song Popularity') #s=sizes, alpha=0.5, label='Song Popularity')

# Add trendline
z = np.polyfit(spotify_youtube_df_cleaned['stream'], spotify_youtube_df_cleaned['views'], 1)
p = np.poly1d(z)
plt.plot(spotify_youtube_df_cleaned['stream'], p(spotify_youtube_df_cleaned['stream']), "r--", label="Trendline")

# Add labels and title
plt.xlabel('Streams (Billions)')  # Label showing units in billions
plt.ylabel('YouTube Views (Billions)')  # Label showing units in billions
plt.title('Streams vs YouTube Views with Likes Represented by Size')
plt.legend()

# Add grid
plt.grid(True)  # This adds a grid to the plot

# Show plot
plt.tight_layout()
plt.show()

### Outliers
As we can see from this scatter plot, a few songs have view or stream counts much greater than the average song. This suggests that keeping these values could possibly skew the data. To correct this, we can either remove the outliers, or log transform the appropriate features.

In [ ]:
# Assuming 'spotify_youtube_df' is your DataFrame containing 'channel' and 'streams' columns
# First, group by channel and sum their total streams
channel_streams = spotify_youtube_df.groupby('channel')['stream'].sum().reset_index()

# Sort the channels by the total streams in descending order and select the top 10
top_10_channels = channel_streams.sort_values(by='stream', ascending=False).head(10)

# Calculate the percentage of total streams for each of the top 10 channels
total_streams = spotify_youtube_df['stream'].sum()
top_10_channels['percent_of_total'] = (top_10_channels['stream'] / total_streams) * 100

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.barh(top_10_channels['channel'], top_10_channels['percent_of_total'], color='skyblue')
plt.xlabel('Percentage of Total Streams')
plt.ylabel('Channel')
plt.title('Top 10 Most Popular Channels by Streams and Their Share of Total Streams')
plt.gca().invert_yaxis()  # Invert y-axis so that the highest value is at the top

# Add percentage labels to each bar
for index, value in enumerate(top_10_channels['percent_of_total']):
    plt.text(value + 0.5, index, f"{value:.2f}%", va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Assuming 'spotify_youtube_df' is your DataFrame containing 'artist' and 'stream' columns
# First, group by artist and sum their total streams
artist_streams = spotify_youtube_df.groupby('artist')['stream'].sum().reset_index()

# Sort the artists by the total streams in descending order and select the top 10
top_10_artists = artist_streams.sort_values(by='stream', ascending=False).head(10)

# Calculate the percentage of total streams for each of the top 10 artists
total_streams = spotify_youtube_df['stream'].sum()
top_10_artists['percent_of_total'] = (top_10_artists['stream'] / total_streams) * 100

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.barh(top_10_artists['artist'], top_10_artists['percent_of_total'], color='lightgreen')
plt.xlabel('Percentage of Total Streams')
plt.ylabel('Artist')
plt.title('Top 10 Most Popular Artists by Streams and Their Share of Total Streams')
plt.gca().invert_yaxis()  # Invert y-axis so that the highest value is at the top

# Add percentage labels to each bar
for index, value in enumerate(top_10_artists['percent_of_total']):
    plt.text(value + 0.5, index, f"{value:.2f}%", va='center')

plt.tight_layout()
plt.show()

## Feature Engineering

Creating new features `power` and `uplift_factor` as averages of existing features.

In [ ]:
# Create new features
spotify_youtube_df_cleaned['power'] = (spotify_youtube_df_cleaned['energy'] + spotify_youtube_df_cleaned['loudness']) / 2
spotify_youtube_df_cleaned['uplift_factor'] = (spotify_youtube_df_cleaned['valence'] + spotify_youtube_df_cleaned['danceability']) / 2

# Models

In [ ]:
# Assume df is your DataFrame and 'target' is your target variable
X = spotify_youtube_df_cleaned[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]  # Replace with your actual feature names
y = spotify_youtube_df_cleaned['engagement']  # Replace with your actual target variable

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Model 1: Linear Regression

In [ ]:
# Create a linear regression model and fit it to the training data.
model = LinearRegression()
model.fit(X_train, y_train)

# Use the model to make predictions on the testing data.
y_pred = model.predict(X_test)

#Evaluate the model's performance using metrics like Mean Squared Error (MSE) and R-squared.
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
#Visualize the Results
'''
plt.scatter(X_test['feature1'], y_test, color='blue')
plt.scatter(X_test['feature1'], y_pred, color='red')
plt.plot(X_test['feature1'], y_pred, color='green')
plt.xlabel('Feature 1')
plt.ylabel('Target')
plt.title('Linear Regression Results')
plt.show()
'''

In [ ]:
from youtubesearchpython import VideosSearch, ResultMode, Video
from tqdm.notebook import tqdm
import datetime
import pprint as pp

def getUploadDate(url):
    video = Video.getInfo(url, mode = ResultMode.json)
    #pp.pprint(video)
    return video['uploadDate'][:10]

#tqdm.pandas()

spotify_youtube_df_cleaned_noerror = spotify_youtube_df_cleaned.copy()
video_dates_df = pd.DataFrame()
bad_rows = []

for i, row in tqdm(spotify_youtube_df_cleaned_noerror.iterrows(), total=spotify_youtube_df_cleaned_noerror.shape[0]):
    try:
        getUploadDate(row.iloc[0])
    except TypeError:
        bad_rows.append(i)
        continue
    else:
        continue   

spotify_youtube_df_cleaned_noerror = spotify_youtube_df_cleaned_noerror.drop(bad_rows)

video_dates_df['id'] = spotify_youtube_df_cleaned['url_youtube'].apply(lambda s: s[-11:])

video_dates_df['id'] = video_dates_df['id'].astype(str)

#spotify_youtube_df_cleaned['url_youtube'][1][-11:]
pp.pprint(video_dates_df)

video_dates_df.to_csv("./debug/video_dates_df.csv")

#for i, row in video_dates_df.iterrows():

#video_dates_df.isnull()
#spotify_youtube_df_cleaned.isnull()
#video_dates_df["upload_date"] = video_dates_df['url'].progress_apply(getUploadDate)
'''
for i, row in video_dates_df.iterrows():
    try:    
        print(f'Row: {i}: {row}')
    except:
        bad_rows.append(i)
    else:
        print(getUploadDate(row.iloc[0]))
'''

In [ ]:
'''
spotify_youtube_df

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm.notebook import tqdm

client_cred_manager = SpotifyClientCredentials(client_id="f7f9bbcca80343a3a9f48cd4f8e8ba28",
                                               client_secret="6588e909ab9a4fab97b7f37a4fb8987b")

sp = spotipy.Spotify(client_credentials_manager=client_cred_manager)

spotify_youtube_df['track_id'] = spotify_youtube_df['uri'].apply(lambda s: s.split(':')[-1])
spotify_youtube_df.head(3)

def get_release_date(track_id):
    #print(f"getting info for {track_id}")
    track_info = sp.track(track_id)
    release_date = track_info['album']['release_date']
    #print(release_date)
    return release_date

tqdm.pandas()

# printing to stdout during lookup too slow, progress bar used instead
spotify_youtube_df['release_date'] = spotify_youtube_df['track_id'].progress_apply(get_release_date)
'''